In [2]:
# Librarys

import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

from sklearn import model_selection, preprocessing, metrics
from sklearn.model_selection import train_test_split
import lightgbm as lgbm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error, r2_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import joblib

from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import (LinearRegression, Ridge, Lasso,ElasticNet)
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor



# Feature Engineering

In [3]:
#Load train and test dataset
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

#delete property
df = df.drop(['property'], axis = 1)
df_test = df_test.drop(['property'], axis = 1)


#keep year and month of selling date
df['date'] = pd.to_datetime(df['date'].astype(str), format='%Y%m%d')
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

df_test['date'] = pd.to_datetime(df_test['date'].astype(str), format='%Y%m%d')
df_test['year'] = df_test['date'].dt.year
df_test['month'] = df_test['date'].dt.month

#delete the old date column
df = df.drop(['date'], axis = 1)
df_test = df_test.drop(['date'], axis = 1)

In [4]:
df=df[df['bedrooms']<11]
df=df.drop(index=67).reset_index(drop=True)

In [5]:
# create new feature= distance from downtown
# downtown seattle lat=47.36217   long=-122.20069

df['lat_to_downtown']=(df['lat']-47.36217).abs()
df['long_to_downtown']=(df['long']-(-122.20069)).abs()

df_test['lat_to_downtown']=(df_test['lat']-47.36217).abs()
df_test['long_to_downtown']=(df_test['long']-(-122.20069)).abs()

df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year,month,lat_to_downtown,long_to_downtown
0,342000.0,3,2.00,1930,11947,1.0,0,0,4,8,...,0,98042,47.3672,-122.151,2200,12825,2014,5,0.00503,0.04969
1,550000.0,4,2.50,1940,10500,1.0,0,0,4,7,...,0,98052,47.6830,-122.114,2200,10500,2014,5,0.32083,0.08669
2,335000.0,2,2.00,1350,2560,1.0,0,0,3,8,...,0,98052,47.6344,-122.107,1790,2560,2014,5,0.27223,0.09369
3,482000.0,4,2.50,2710,35868,2.0,0,0,3,9,...,0,98038,47.3750,-122.022,2780,36224,2014,5,0.01283,0.17869
4,463000.0,3,1.75,1710,7320,1.0,0,0,3,7,...,0,98155,47.7512,-122.281,2260,8839,2014,5,0.38903,0.08031


In [6]:
df.columns

Index(['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'year', 'month', 'lat_to_downtown',
       'long_to_downtown'],
      dtype='object')

In [7]:
df['mean_sqft_lot'] = df.groupby(by=['zipcode','grade'])['sqft_lot'].transform('mean')
df_test['mean_sqft_lot'] = df_test.groupby(by=['zipcode','grade'])['sqft_lot'].transform('mean')

df['mean_sqft_living15'] = df.groupby(by=['zipcode','grade'])['sqft_living15'].transform('mean')
df_test['mean_sqft_living15'] = df_test.groupby(by=['zipcode','grade'])['sqft_living15'].transform('mean')


df['mean_sqft_lot15'] = df.groupby(by=['zipcode','grade'])['sqft_lot15'].transform('mean')
df_test['mean_sqft_lot15'] = df_test.groupby(by=['zipcode','grade'])['sqft_lot15'].transform('mean')

df['mean_sqft_living15'] = df.groupby(by=['zipcode','grade'])['sqft_living15'].transform('mean')
df_test['mean_sqft_living15'] = df_test.groupby(by=['zipcode','grade'])['sqft_living15'].transform('mean')

df['mean_sqft_sqft_above'] = df.groupby(by=['zipcode','grade'])['sqft_above'].transform('mean')
df_test['mean_sqft_sqft_above'] = df_test.groupby(by=['zipcode','grade'])['sqft_above'].transform('mean')

df['mean_grade'] = df.groupby(by=['zipcode'])['grade'].transform('mean')
df_test['mean_grade'] = df_test.groupby(by=['zipcode'])['grade'].transform('mean')


df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,sqft_lot15,year,month,lat_to_downtown,long_to_downtown,mean_sqft_lot,mean_sqft_living15,mean_sqft_lot15,mean_sqft_sqft_above,mean_grade
0,342000.0,3,2.00,1930,11947,1.0,0,0,4,8,...,12825,2014,5,0.00503,0.04969,17773.796875,2292.250000,14467.500000,2227.625000,7.416327
1,550000.0,4,2.50,1940,10500,1.0,0,0,4,7,...,10500,2014,5,0.32083,0.08669,9034.290323,1706.129032,8434.564516,1337.870968,8.186047
2,335000.0,2,2.00,1350,2560,1.0,0,0,3,8,...,2560,2014,5,0.27223,0.09369,9605.580357,2204.017857,9248.107143,1924.732143,8.186047
3,482000.0,4,2.50,2710,35868,2.0,0,0,3,9,...,36224,2014,5,0.01283,0.17869,31760.219512,2779.024390,25376.658537,2885.390244,7.600733
4,463000.0,3,1.75,1710,7320,1.0,0,0,3,7,...,8839,2014,5,0.38903,0.08031,9482.690909,1630.463636,9240.445455,1394.181818,7.250000


In [8]:
df['price']=np.log(1 + df['price'])

df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,sqft_lot15,year,month,lat_to_downtown,long_to_downtown,mean_sqft_lot,mean_sqft_living15,mean_sqft_lot15,mean_sqft_sqft_above,mean_grade
0,12.742569,3,2.00,1930,11947,1.0,0,0,4,8,...,12825,2014,5,0.00503,0.04969,17773.796875,2292.250000,14467.500000,2227.625000,7.416327
1,13.217675,4,2.50,1940,10500,1.0,0,0,4,7,...,10500,2014,5,0.32083,0.08669,9034.290323,1706.129032,8434.564516,1337.870968,8.186047
2,12.721889,2,2.00,1350,2560,1.0,0,0,3,8,...,2560,2014,5,0.27223,0.09369,9605.580357,2204.017857,9248.107143,1924.732143,8.186047
3,13.085701,4,2.50,2710,35868,2.0,0,0,3,9,...,36224,2014,5,0.01283,0.17869,31760.219512,2779.024390,25376.658537,2885.390244,7.600733
4,13.045484,3,1.75,1710,7320,1.0,0,0,3,7,...,8839,2014,5,0.38903,0.08031,9482.690909,1630.463636,9240.445455,1394.181818,7.250000


In [9]:
df['sqft_living']=np.log(1 + df['sqft_living'])
df['sqft_living15']=np.log(1 + df['sqft_living15'])
df['sqft_lot']=np.log(1 + df['sqft_lot'])
df['sqft_lot15']=np.log(1 + df['sqft_lot15'])

df_test['sqft_living']=np.log(1 + df_test['sqft_living'])
df_test['sqft_living15']=np.log(1 + df_test['sqft_living15'])
df_test['sqft_lot']=np.log(1 + df_test['sqft_lot'])
df_test['sqft_lot15']=np.log(1 + df_test['sqft_lot15'])

df['mean_sqft_lot'] = df.groupby(by=['zipcode','grade'])['sqft_lot'].transform('mean')
df_test['mean_sqft_lot'] = df_test.groupby(by=['zipcode','grade'])['sqft_lot'].transform('mean')

df['mean_sqft_living15'] = df.groupby(by=['zipcode','grade'])['sqft_living15'].transform('mean')
df_test['mean_sqft_living15'] = df_test.groupby(by=['zipcode','grade'])['sqft_living15'].transform('mean')


df['mean_sqft_lot15'] = df.groupby(by=['zipcode','grade'])['sqft_lot15'].transform('mean')
df_test['mean_sqft_lot15'] = df_test.groupby(by=['zipcode','grade'])['sqft_lot15'].transform('mean')

df['mean_sqft_living15'] = df.groupby(by=['zipcode','grade'])['sqft_living15'].transform('mean')
df_test['mean_sqft_living15'] = df_test.groupby(by=['zipcode','grade'])['sqft_living15'].transform('mean')

df['mean_sqft_sqft_above'] = df.groupby(by=['zipcode','grade'])['sqft_above'].transform('mean')
df_test['mean_sqft_sqft_above'] = df_test.groupby(by=['zipcode','grade'])['sqft_above'].transform('mean')

df['mean_grade'] = df.groupby(by=['zipcode'])['grade'].transform('mean')
df_test['mean_grade'] = df_test.groupby(by=['zipcode'])['grade'].transform('mean')

In [10]:
#Split train dataset to train and test

X_train,X_test,y_train,y_test=train_test_split(df.drop('price',axis=1),df['price'],test_size=0.1, random_state=3)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(8997, 27)
(8997,)
(1000, 27)
(1000,)


# LightGBM

In [11]:
#Create train and validation datasets for LightGBM classifier
lgbm_train = lgbm.Dataset(X_train, y_train)
lgbm_eval = lgbm.Dataset(X_test, y_test, reference=lgbm_train)

In [12]:
# Create grid parameters for tuning LightGBM

gridParams = {
    "max_depth": [-1],
    'learning_rate': [0.04,0.045,0.05,0.055],
    'n_estimators': [200,500],
    'num_leaves': [20,25,50],
    'boosting_type' : ['gbdt'],
    'feature_fraction': [0.7,0.9],
    'bagging_fraction' :[0.9],
    'bagging_freq' :[10]  
    }

params = {'random_state' : 10,
          'objective': 'regression',
          'min_data_in_leaf': 20,      
          'metric': [12,'rmse']
         }

In [13]:
# Create regressor 

clf = lgbm.LGBMRegressor(**params)


# Create the grid
grid = GridSearchCV(clf, gridParams,
                    verbose=2,
                    cv=5,
                    n_jobs=2,
                    return_train_score = True)
# Run the grid
grid.fit(X_train, y_train)


# Select the best values obtained in tuning part
params['learning_rate'] = grid.best_params_['learning_rate']
params['num_leaves'] = grid.best_params_['num_leaves']
params['learning_rate'] = grid.best_params_['learning_rate']
params['max_depth']= grid.best_params_['max_depth']
params['learning_rate']= grid.best_params_['learning_rate']
params['num_leaves']= grid.best_params_['num_leaves']
params['boosting_type']= grid.best_params_['boosting_type']
params['bagging_fraction']= grid.best_params_['bagging_fraction']
params['bagging_freq']= grid.best_params_['bagging_freq']
params['feature_fraction']= grid.best_params_['feature_fraction']


grid.best_params_

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:   54.4s
[Parallel(n_jobs=2)]: Done 158 tasks      | elapsed:  3.7min
[Parallel(n_jobs=2)]: Done 240 out of 240 | elapsed:  6.1min finished


{'bagging_fraction': 0.9,
 'bagging_freq': 10,
 'boosting_type': 'gbdt',
 'feature_fraction': 0.7,
 'learning_rate': 0.045,
 'max_depth': -1,
 'n_estimators': 500,
 'num_leaves': 20}

In [14]:
df.columns

Index(['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'year', 'month', 'lat_to_downtown',
       'long_to_downtown', 'mean_sqft_lot', 'mean_sqft_living15',
       'mean_sqft_lot15', 'mean_sqft_sqft_above', 'mean_grade'],
      dtype='object')

In [15]:
#train with tuned parameters

evaluation_results = {}
clf_tuned = lgbm.train(params,
                 lgbm_train,
                 valid_sets=[lgbm_train,lgbm_eval], 
                 valid_names=['Train', 'Test'],
                 categorical_feature=[13],
                 evals_result=evaluation_results,
                 num_boost_round=1500,
                 early_stopping_rounds=300,
                 verbose_eval=20
                )

clf_tuned.save_model('lgbm_model.txt', num_iteration=clf_tuned.best_iteration) 

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1188: UserWarning:

categorical_feature in Dataset is overridden.
New categorical_feature is [13]

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:742: UserWarning:

categorical_feature in param dict is overridden.



Training until validation scores don't improve for 300 rounds.
[20]	Train's rmse: 0.294362	Test's rmse: 0.284975
[40]	Train's rmse: 0.209612	Test's rmse: 0.207277
[60]	Train's rmse: 0.178098	Test's rmse: 0.182373
[80]	Train's rmse: 0.164767	Test's rmse: 0.173257
[100]	Train's rmse: 0.157371	Test's rmse: 0.169252
[120]	Train's rmse: 0.152296	Test's rmse: 0.167629
[140]	Train's rmse: 0.148428	Test's rmse: 0.166214
[160]	Train's rmse: 0.145027	Test's rmse: 0.165043
[180]	Train's rmse: 0.142264	Test's rmse: 0.164448
[200]	Train's rmse: 0.139797	Test's rmse: 0.16366
[220]	Train's rmse: 0.137512	Test's rmse: 0.163494
[240]	Train's rmse: 0.135498	Test's rmse: 0.163032
[260]	Train's rmse: 0.133505	Test's rmse: 0.162704
[280]	Train's rmse: 0.131678	Test's rmse: 0.162339
[300]	Train's rmse: 0.129903	Test's rmse: 0.162199
[320]	Train's rmse: 0.12825	Test's rmse: 0.162278
[340]	Train's rmse: 0.126646	Test's rmse: 0.1621
[360]	Train's rmse: 0.125077	Test's rmse: 0.161793
[380]	Train's rmse: 0.12357

In [16]:
#Important features identified by LightGBM
clf_tuned = lgbm.Booster(model_file='lgbm_model.txt')

important_features = pd.DataFrame({'features': clf_tuned.feature_name(),
                            'importance': clf_tuned.feature_importance()}).sort_values('importance', ascending=False)
important_features

,features,importance
13,zipcode,1382
11,yr_built,1089
3,sqft_lot,1087
16,sqft_living15,1027
2,sqft_living,979
14,lat,920
9,sqft_above,907
17,sqft_lot15,859
21,long_to_downtown,774
15,long,706


In [17]:
df.columns

Index(['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'year', 'month', 'lat_to_downtown',
       'long_to_downtown', 'mean_sqft_lot', 'mean_sqft_living15',
       'mean_sqft_lot15', 'mean_sqft_sqft_above', 'mean_grade'],
      dtype='object')

In [18]:
# Scores obtained by LightGBM
def adjustedR2(r2,n,k):
    return r2-(k-1)/(n-k)*(1-r2)

lgbm_preds=clf_tuned.predict(X_test)
print('Mean absolute error = \t {}'.format(np.round(mean_absolute_error(y_test, lgbm_preds), 2 )))
print('RMSE = \t {}'.format(np.round(np.sqrt(mean_squared_error(y_test, lgbm_preds)),5 )))
print('R2 = \t {}'.format(np.round(r2_score(y_test, lgbm_preds), 3 )))
print('R2_adjusted = \t {}'.format(np.round(adjustedR2(r2_score(y_test, lgbm_preds),X_test.shape[0],X_test.shape[1]), 3)))

Mean absolute error = 	 0.11
RMSE = 	 0.16049
R2 = 	 0.903
R2_adjusted = 	 0.901


In [19]:
#Sumbmission of test prediction obtained by LightGBM
test_id = df_test['id']
df_test_lgbm=df_test.drop('id',axis=1)
preds_test = clf_tuned.predict(df_test_lgbm)
LightGBM_to_ens=preds_test
preds_test=np.exp(preds_test)-1
test_id = pd.DataFrame(test_id)
pred_df = pd.DataFrame(preds_test)

LightGBM_preds = test_id.join(pred_df)
LightGBM_preds.columns = ['id', 'price']

LightGBM_preds.to_csv("LightGBM.csv", header=True, index=False)

In [20]:
LightGBM_preds.head()

,id,price
0,1,3.090505e+05
1,2,2.002700e+06
2,3,5.144256e+05
3,4,4.044886e+05
4,5,4.091814e+05


In [21]:

X_train.columns

Index(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'year', 'month', 'lat_to_downtown',
       'long_to_downtown', 'mean_sqft_lot', 'mean_sqft_living15',
       'mean_sqft_lot15', 'mean_sqft_sqft_above', 'mean_grade'],
      dtype='object')

In [22]:
# Create grid parameters for tuning LightGBM

gridParams = {
    'learning_rate': [0.045,0.05,0.055],
    'iterations': [1000],
    'depth': [3,4],
    }

cat_feature_names=['zipcode']
categorical_features_indices = np.where(X_train.columns.isin(cat_feature_names))[0]
params = {'random_state' : 10,      
          'loss_function': 'RMSE',
          'cat_features': categorical_features_indices
         }

In [ ]:
# Create regressor 

catb = CatBoostRegressor(**params)


# Create the grid
grid = GridSearchCV(catb, gridParams,
                    verbose=2,
                    cv=5,
                    n_jobs=2,
                    return_train_score = True)
# Run the grid
grid.fit(X_train, y_train)


# Select the best values obtained in tuning part
params['learning_rate'] = grid.best_params_['learning_rate']
params['depth']= grid.best_params_['depth']
params['iterations']= grid.best_params_['iterations']


grid.best_params_

# Catboost

In [ ]:
# cat_feature_names=['waterfront','view','grade','condition','yr_built','month','zipcode']
# cat_feature_names=['grade','condition','zipcode']

cat_feature_names=['zipcode']
categorical_features_indices = np.where(X_train.columns.isin(cat_feature_names))[0]


catb=CatBoostRegressor(iterations=20000, depth=3, cat_features=categorical_features_indices, learning_rate=0.05, loss_function='RMSE',early_stopping_rounds=3000)
catb.fit(X_train, y_train,eval_set=(X_test, y_test),plot=True)

In [29]:
#save model
catb.save_model('catb')

In [30]:
#load model
catb = CatBoostRegressor()
catb.load_model("catb")

In [31]:
# Scores obtained by LightGBM
def adjustedR2(r2,n,k):
    return r2-(k-1)/(n-k)*(1-r2)

catb_preds=catb.predict(X_test)
print('Mean absolute error = \t {}'.format(np.round(mean_absolute_error(y_test, catb_preds), 2 )))
print('RMSE = \t {}'.format(np.round(np.sqrt(mean_squared_error(y_test, catb_preds)),5 )))
print('R2 = \t {}'.format(np.round(r2_score(y_test, catb_preds), 3 )))
print('R2_adjusted = \t {}'.format(np.round(adjustedR2(r2_score(y_test, catb_preds),X_test.shape[0],X_test.shape[1]), 3)))

Mean absolute error = 	 0.12
RMSE = 	 0.16543
R2 = 	 0.897
R2_adjusted = 	 0.895


In [32]:
#Sumbmission of test prediction obtained by LightGBM
test_id = df_test['id']
df_test_catb=df_test.drop('id',axis=1)
preds_test = catb.predict(df_test_catb)
Catboost_to_ens=preds_test
preds_test=np.exp(preds_test)-1
test_id = pd.DataFrame(test_id)
pred_df = pd.DataFrame(preds_test)

Catboost_preds = test_id.join(pred_df)
Catboost_preds.columns = ['id', 'price']

Catboost_preds.to_csv("Catboost.csv", header=True, index=False)

# XGboost

In [33]:
#cv for xgboost

param_grid = {
    "xgbrg__n_estimators": [500],
    "xgbrg__learning_rate": [0.17,0.15],
    'max_depth': [3,4,5],
    'subsample': [0.75,0.8,0.85]
}

fit_params = {"xgbrg__eval_set": [(X_test, y_test)], 
              "xgbrg__early_stopping_rounds": 50, 
              "xgbrg__verbose": False,
              "tree_method ": "hist",
              "grow_policy" : "lossguide"} 



#xgbr = lgbm.LGBMRegressor(**fit_params)
xgbr = xgb.XGBRegressor(**fit_params)

searchCV = GridSearchCV(xgbr, cv=10, param_grid=param_grid,verbose=2,
                    n_jobs=2,scoring= 'neg_mean_squared_error')

searchCV.fit(X_train, y_train) 

Fitting 10 folds for each of 18 candidates, totalling 180 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:   58.9s
[Parallel(n_jobs=2)]: Done 158 tasks      | elapsed:  4.3min
[Parallel(n_jobs=2)]: Done 180 out of 180 | elapsed:  4.9min finished


GridSearchCV(cv=10, error_score=nan,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, grow_policy='lossguide',
                                    importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_...
                                    tree_method ='hist',
                                    validate_parameters=False, verbosity=None,
                                    xgbrg__early_stopping_rounds=50, ...),
             iid='deprecated', n_jobs=2,
             param_grid={'max_depth': [3

In [34]:
#training xgboost with the cv winner
xgb_pars={}
xgb_pars['xgbrg__learning_rate']=searchCV.best_params_['xgbrg__learning_rate']
xgb_pars['xgbrg__n_estimators']=searchCV.best_params_['xgbrg__n_estimators']
xgb_pars['max_depth']=searchCV.best_params_['max_depth']
xgb_pars['subsample']=searchCV.best_params_['subsample']
xgb_pars

{'xgbrg__learning_rate': 0.17,
 'xgbrg__n_estimators': 500,
 'max_depth': 4,
 'subsample': 0.85}

In [35]:
searchCV.best_params_

{'max_depth': 4,
 'subsample': 0.85,
 'xgbrg__learning_rate': 0.17,
 'xgbrg__n_estimators': 500}

In [36]:
xgb_pars={}
xgb_pars={**fit_params,**searchCV.best_params_}

In [37]:
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)

In [38]:
eval_set = [(dtrain, 'train'),(dtest, 'eval')]
num_round=500

xgb_tuned = xgb.train(xgb_pars, dtrain , num_round , eval_set , early_stopping_rounds=250,verbose_eval=15)

#save model
joblib.dump(xgb_tuned,'xgb_model') 

[19:53:15] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { tree_method , xgbrg__early_stopping_rounds, xgbrg__eval_set, xgbrg__learning_rate, xgbrg__n_estimators, xgbrg__verbose } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:8.78995	eval-rmse:8.81177
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 250 rounds.
[15]	train-rmse:0.19001	eval-rmse:0.19261
[30]	train-rmse:0.16106	eval-rmse:0.17660
[45]	train-rmse:0.15031	eval-rmse:0.17288
[60]	train-rmse:0.14215	eval-rmse:0.17230
[75]	train-rmse:0.13717	eval-rmse:0.17172
[90]	train-rmse:0.13186	eval-rmse:0.17100
[105]	train-rmse:0.12827	eval-rmse:0.17193
[120]	train-rmse:0.12433

['xgb_model']

In [39]:
# Scores obtained by XGBoost

#load saved model
xgb_tuned = joblib.load('xgb_model')
best_iteration = xgb_tuned.best_iteration


def adjustedR2(r2,n,k):
    return r2-(k-1)/(n-k)*(1-r2)


xgb_preds=xgb_tuned.predict(dtest,ntree_limit=best_iteration)

print('Mean absolute error = \t {}'.format(np.round(mean_absolute_error(y_test, xgb_preds), 2 )))
print('RMSE = \t {}'.format(np.round(np.sqrt(mean_squared_error(y_test, xgb_preds)),5 )))
print('R2 = \t {}'.format(np.round(r2_score(y_test, xgb_preds), 3 )))
print('R2_adjusted = \t {}'.format(np.round(adjustedR2(r2_score(y_test, xgb_preds),X_test.shape[0],X_test.shape[1]), 3)))

Mean absolute error = 	 0.12
RMSE = 	 0.16992
R2 = 	 0.892
R2_adjusted = 	 0.889


In [40]:
#Sumbmission of test prediction obtained by Xgboost
test_id = df_test['id']
df_test_xgb =df_test.drop('id',axis=1)
df_test_xgb2=xgb.DMatrix(df_test_xgb)
preds_test = xgb_tuned.predict(df_test_xgb2)
XGB_to_ens=preds_test
preds_test=np.exp(preds_test)-1
test_id = pd.DataFrame(test_id)
pred_df = pd.DataFrame(preds_test)

XGB_preds = test_id.join(pred_df)
XGB_preds.columns = ['id', 'price']

XGB_preds.to_csv("XGB.csv", header=True, index=False)

# Ensembling

In [41]:
test_preds = pd.DataFrame(
    {'lgbm_preds': LightGBM_to_ens,
     'xgb_preds': XGB_to_ens,
    'catb_preds': Catboost_to_ens
#     'slm_preds': SLR_to_ens,
#     'elastic_poly_preds': EP_to_ens
    })
test_preds.head()

,lgbm_preds,xgb_preds,catb_preds
0,12.641263,12.619909,12.650058
1,14.510007,14.517980,14.509011
2,13.150808,13.063245,13.140725
3,12.910381,12.849273,12.942562
4,12.921916,13.084271,12.984057


In [42]:
#Ensebling test data

all_preds=pd.DataFrame(y_test)
all_preds['lgbm_preds']=lgbm_preds
all_preds['xgb_preds']=xgb_preds
all_preds['catb_preds']=catb_preds
#all_preds['slm_preds']=slr_preds
#all_preds['elastic_poly_preds']=Elastic_poly_preds

lm = linear_model.LinearRegression(fit_intercept=False)
# lm = linear_model.LinearRegression()
lm.fit(all_preds.drop('price',axis=1),all_preds['price'])
print('Intercept: {}'.format(lm.intercept_))
print('Coefficients: {}'.format(lm.coef_))
ens_preds = lm.predict(all_preds.drop('price',axis=1))


#Ensebling submission
test_id = df_test['id']
all_ens = pd.DataFrame(test_id)
test_preds_ens=lm.predict(test_preds)
all_ens['price']=np.exp(test_preds_ens)-1
                                          
all_ens.to_csv("Ensemble.csv", header=True, index=False)

Intercept: 0.0
Coefficients: [0.58330312 0.13284364 0.28384157]


In [43]:
print('Mean absolute error = \t {}'.format(np.round(mean_absolute_error(y_test, ens_preds), 2 )))
print('RMSE = \t {}'.format(np.round(np.sqrt(mean_squared_error(y_test, ens_preds)),5 )))
print('R2 = \t {}'.format(np.round(r2_score(y_test, ens_preds), 3 )))
print('R2_adjusted = \t {}'.format(np.round(adjustedR2(r2_score(y_test, ens_preds),X_test.shape[0],X_test.shape[1]), 3)))

Mean absolute error = 	 0.11
RMSE = 	 0.15849
R2 = 	 0.906
R2_adjusted = 	 0.903


In [44]:
# Plotting all methods' predictions

In [48]:
x= np.arange(50)

trace0 = go.Scatter(
    y = np.exp(catb_preds)-1,
    x = x, 
    mode = 'lines', 
    name = 'catboost',
    marker = dict(
    color = 'rgb(240, 150, 50)')
)

trace1 = go.Scatter(
    y = np.exp(lgbm_preds)-1,
    x = x, 
    mode = 'lines', 
    name = 'lgbm',
    line = dict(
        color = 'rgb(90, 70, 150)',
    )
)
    
trace2 = go.Scatter(
    y = np.exp(xgb_preds)-1 ,
    x = x, 
    mode = 'lines', 
    name = 'xgboost',
    line = dict(
        color = 'rgb(150, 60, 120)',
    )
)


trace3 = go.Scatter(
    y = np.exp(ens_preds)-1,
    x = x, 
    mode = 'lines', 
    name = 'ensemble',
    line = dict(
        color = 'rgb(200, 115, 147)',
    )
)

trace4 = go.Scatter(
    y = np.exp(y_test)-1,
    x = x, 
    mode = 'lines', 
    name = 'actual',
    line = dict(
        color = 'rgb(100, 120, 160)',
        dash = 'dot'
    )
)


layout = go.Layout(
    xaxis = dict(title = 'Index'), 
    yaxis = dict(title = 'Price')
)

figure = go.Figure(data = [trace0,trace1,trace2,trace3,trace4], layout = layout)
iplot(figure)